# 2025 Update on Mortality of Switzerland and Other Countries

In 2021, I wrote a blog post about [Swiss mortality](https://lorentzen.ch/index.php/2021/02/19/swiss-mortality/) and it turned out to be among the most read posts I have written.
Four years later, I think it's a good idea to give an update as we have 4 years more observations.

## 1. Human Mortality Database

Data Sources:
- [Human Mortality Database](https://www.mortality.org/)</br>
  [Short-Term Mortality Fluctuations](https://www.mortality.org/Data/STMF)</br>
  Data: https://www.mortality.org/File/GetDocument/Public/STMF/Outputs/stmf.csv

Code run 5 July 2025.

In [1]:
from datetime import datetime
import polars as pl
import altair as alt

# https://altair-viz.github.io/user_guide/large_datasets.html
alt.data_transformers.enable("vegafusion")

df_original = pl.read_csv(
    "https://www.mortality.org/File/GetDocument/Public/STMF/Outputs/stmf.csv",
    skip_rows=2,
    # Help polars a bit:
    schema_overrides={
        "D65_74": pl.Float64,
        "D75_84": pl.Float64,
        "D85p": pl.Float64,
        "DTotal": pl.Float64,
    },
)

In [2]:
df_original["Week"].min(), df_original["Week"].max()

(1, 53)

In [3]:
df_mortality = df_original.filter(
    # Select country of interest and only "both" sexes.
    # Note: Germany "DEUTNP" and "USA" have short time series.
    pl.col("CountryCode").is_in(["CAN", "CHE", "FRATNP", "GBRTENW", "SWE"]),
    pl.col("Sex") == pl.lit("b"),
).with_columns(
    # Change to ISO-3166-1 ALPHA-3 codes
    CountryCode=pl.col("CountryCode").replace(
        {"FRATNP": "FRA", "GBRTENW": "England & Wales"},
    ),
    # Create population pro rata temporis (exposure) to ease aggregation
    Population=pl.col("DTotal") / pl.col("RTotal"),
).with_columns(
    # We think that the data uses ISO 8601 week dates and we set the weekday
    # to 1, i.e., Monday.
    Date=(
        pl.col("Year").cast(pl.String)
        + "-W" + pl.col("Week").cast(pl.String).str.zfill(2)
        + "-1"
    ).str.to_date(format="%G-W%V-%u")
)

In [4]:
chart = (
    alt.Chart(
        df_mortality.filter(pl.col("Year") <= 2024)
        # The Covid-19 peaks in 2020 are better seen on weekly resolution.
        .group_by("Year", "CountryCode")
        .agg(pl.col("Population").sum(), pl.col("DTotal").sum())
        .with_columns(
            CDR=pl.col("DTotal") / pl.col("Population"),
        )
    )
    .mark_line()
    .encode(
        x="Year:T",
        y=alt.Y("CDR:Q", scale=alt.Scale(zero=False)),
        color="CountryCode:N",
    )
    .properties(title="Crude Death Rate per Year")
    .interactive()
)
# chart.save("crude_death_rate.html")
chart

alt.Chart(...)

Let's have an additional plot of Switzerland only, but with weekly CDR to see the seasonal fluctuations. 

In [5]:
chart = (
    alt.Chart(
        df_mortality.filter(
            pl.col("CountryCode") <= pl.lit("CHE"),
            # Last 12 years
            pl.col("Year") > pl.col("Year").max() - 12,
        ).with_columns(
            CDR=pl.col("DTotal") / pl.col("Population"),
        )
    )
    .mark_line()
    .encode(
        x="Date:T",
        y=alt.Y("CDR:Q", scale=alt.Scale(zero=True)),
    )
    .properties(
        title="Crude Death Rate per Week for Switzerland",
        width=400,  # default 300
    )
    .interactive()
)
# chart.save("crude_death_rate_per_week.html")
chart

alt.Chart(...)

## 2. Federal Statistical Office of Switzerland
We extend our analysis for Switzerland with other data sources in order to look over the past 100 years.

Data sources:
- [Todesfälle nach Monat und Sterblichkeit seit 1803](https://www.bfs.admin.ch/asset/de/px-x-0102020206_111)</br>
  BFS-Nummer px-x-0102020206_111
- [Ständige Wohnbevölkerung nach Geschlecht und Alter, 1860-2023](https://www.bfs.admin.ch/asset/de/px-x-0102030000_101)</br>
  BFS-Nummer 	px-x-0102030000_101

In [6]:
from contextlib import contextmanager
import locale

from pyaxis import pyaxis


@contextmanager
def localized(code):
    old_code, old_encoding = locale.getlocale()
    if old_code is None:
        old_code = "de_DE"
    locale.setlocale(locale.LC_ALL, code)
    yield

In [7]:
# px-x-0102020206_111
px = pyaxis.parse(
    "https://dam-api.bfs.admin.ch/hub/api/dam/assets/32506839/master",
    encoding="UTF-8",
)
df_death_original = pl.from_pandas(px["DATA"])

# px-x-0102030000_101
px = pyaxis.parse(
    "https://dam-api.bfs.admin.ch/hub/api/dam/assets/32207866/master",
    encoding="UTF-8",
)
df_popul_original = pl.from_pandas(px["DATA"])

Multilingual PX file
Multilingual PX file


In [ ]:
# Preprocess deaths data
df_death = df_death_original.filter(
    pl.col("Demografisches Merkmal und Indikator").str.contains("Todesfälle im")
).with_columns(
    Year=pl.col("Jahr").str.to_integer(),
    Monat_DE=pl.col("Demografisches Merkmal und Indikator").str.split(" ").list.last(),
    # Note: This only creates Null values for Jahr <= 1900.
    Deaths=pl.col("DATA").str.to_integer(strict=False),
).filter(pl.col("Year") >= 1901)

# Datetime localization is not supported in polars, see
# https://github.com/pola-rs/polars/issues/12341.
# So we switch to calling python datetime (instead of installing polars-xdt).
with localized("de_DE.UTF-8"):
    df_death = df_death.with_columns(
        Date=(pl.col("Monat_DE") + " " + pl.col("Year").cast(pl.String))
        .map_elements(
            lambda x: datetime.strptime(x, "%B %Y").date(), return_dtype=pl.Date
        )
    )

df_death = df_death.group_by("Year").agg(Deaths=pl.col("Deaths").sum())


# Preprocess population data
df_popul = df_popul_original.filter(
    pl.col("Geschlecht") == "Geschlecht - Total",
    pl.col("Alter") == "Alter - Total",
).with_columns(
    Year=pl.col("Jahr").str.to_integer(),
    Population=pl.col("DATA").cast(pl.Float64)
).filter(pl.col("Year") >= 1901)

# Merging
df = df_death.join(
    df_popul,
    on="Year",
    how="inner",
).with_columns(
    Date=pl.date(year=pl.col("Year"), month=12, day=31),
    CDR=pl.col("Deaths") / pl.col("Population"),
).select(
    "Year", "Date", "Population", "Deaths", "CDR"
)

print(f"Year from {df["Year"].min()} to {df["Year"].max()}")

Year from 1901 to 2023


/var/folders/1c/ynyjslg54654183kt57yfsdw0000gn/T/ipykernel_2177/2204083038.py:16: PolarsInefficientMapWarning: 
Expr.map_elements is significantly slower than the native expressions API.
Only use if you absolutely CANNOT implement your logic otherwise.
Replace this expression...
  - pl.col("Monat_DE").map_elements(lambda x: ...)
with this one instead:
  + pl.col("Monat_DE").str.to_datetime(format="%B %Y").dt.date()

  .map_elements(


In [9]:
# Compare the 2 different sources
df_compare = df.join(
    df_mortality.filter(
        pl.col("CountryCode") == pl.lit("CHE")
    ).group_by("Year").agg(
        pl.col("Population").sum(), pl.col("DTotal").sum()
    ).with_columns(
        CDR=pl.col("DTotal") / pl.col("Population"),
    ),
    on="Year",
    how="inner",
    suffix="_right",
).with_columns(
    rel_diff=pl.col("CDR") / pl.col("CDR_right") - 1
)
print(
    "Largest relative differences (negative, positive): ",
    df_compare["rel_diff"].min(),
    df_compare["rel_diff"].max(),
)

Largest relative differences (negative, positive):  -0.004597190612424251 0.004854808915297104


This is a quite impressive match as we used two very different data sources with different time resolutions.
As comparison, the 2021 analysis had (-0.0056, 0.0042).

In [10]:
# Append year 2024 from Human Mortality data
df_full = pl.concat([
    df.drop("Date"),
    df_mortality.filter(
        pl.col("CountryCode") == pl.lit("CHE"), pl.col("Year") == 2024
    ).select(
        "Year", "Population", pl.col("DTotal").alias("Deaths"),
        (pl.col("DTotal") / pl.col("Population")).alias("CDR"),
    )
], how="vertical_relaxed")

In [11]:
base = alt.Chart(df).encode(
    alt.X("Year:T", scale=alt.Scale(domain=["1900-01-01", "2025-01-01"]))
)

line_A = base.mark_line(color="#5276A7").encode(
    y=alt.Y("CDR:Q", scale=alt.Scale(zero=False), axis=alt.Axis(titleColor="#5276A7"))
)

line_B = base.mark_line(color="#F18727", strokeDash=[1]).encode(
    y=alt.Y("Population:Q", axis=alt.Axis(titleColor="#F18727"))
)

chart = (
    alt.layer(line_A, line_B)
    .resolve_scale(y="independent")
    .properties(title="Crude Death Rate per Year of Switzerland")
    .interactive()
)
# chart.save("swiss_mortality_chart.html")
chart

alt.LayerChart(...)